# 🚆Train
지하철 데이터 전처리

### TODO

1. 지하철 정류장 데이터 
   - [X] 데이터 불러오기, 필요없는 로우 및 컬럼 일차 삭제  
   - [X] 결측값 확인
     - [X] 위경도 결측값 분석
     - [X] 결측값 채워진 데이터셋 구하기
     - [X] 노선명 통일
     - [X] 역사명 통일
   - [X] 두 데이터셋 하나로 결합
     - [X] 위경도 이상값 확인 
     - [X] 위경도 결측값 재확인
     - [ ] 역번호 이상값 확인
   - [ ] 서울 및 각 역 위치 지도에 시각화
   - [ ] 이상값 확인
   - [ ] 수도권 바깥 데이터 삭제 
   - [ ] 필요없는 로우 및 컬럼 삭제 
   - [ ] 환승역, 종점, 분기역 정보 가공
2. 지하철 시간표 데이터
   - [ ] 데이터 불러오기, 필요없는 로우 및 컬럼 일차 삭제  
   - [ ] 결측값 및 이상값 확인
   - [ ] 방향별 종점별 막차 시간표 정보로 가공

### 0. import

In [299]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from tqdm import trange, notebook

### 1. 지하철 정류장 데이터

##### 1-a. 데이터 불러오기, 필요없는 로우 및 컬럼 일차 삭제  

In [300]:
# 데이터 읽어오기
station = pd.read_csv('train_station.csv')

# 확실히 필요없는 컬럼 제거
station = station.drop(columns=['영문역사명', '한자역사명', '운영기관명', '역사도로명주소', '역사전화번호','데이터기준일자'])

# 확실히 필요없는 행 제거
# # 인천 1~2호선
# station = station[(station['노선번호'].str.slice(stop=3) != 'S28')]
# 인천 자기부상철도 -> 휴업 상태
station = station[(station['노선번호'] != 'S28M1')]
# 대전
station = station[(station['노선번호'].str.slice(start=1, stop=3) != '30')]
# 대구 1~3호선
station = station[(station['노선번호'].str.slice(start=1, stop=3) != '27')]
# 부산 1~4호선, 김해선
station = station[(station['노선번호'].str.slice(start=1, stop=3) != '26') & (station['노선번호'] != 'L48B1')]
# 광주
station = station[(station['노선번호'].str.slice(start=1, stop=3) != '29')]

# # 나중을 위해 역번호 저장
# station_id = station['역번호']

station = station.reset_index(drop=True)
station

,역번호,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,역위도,역경도
0,3110,계양,S2801,인천지하철 1호선,환승역,S2801+I28A1,공항철도,37.571539,126.736319
1,3111,귤현,S2801,인천지하철 1호선,일반역,-,-,37.566362,126.742498
2,3112,박촌,S2801,인천지하철 1호선,일반역,-,-,37.553525,126.744946
3,3113,임학,S2801,인천지하철 1호선,일반역,-,-,37.545058,126.738642
4,3114,계산,S2801,인천지하철 1호선,일반역,-,-,37.543243,126.728436
...,...,...,...,...,...,...,...,...,...
748,4811,달미역,I41WS,서해선,일반역,NaN,NaN,37.349320,126.808218
749,4812,선부역,I41WS,서해선,일반역,NaN,NaN,37.333908,126.810977
750,4813,초지역,I41WS,서해선,환승역,"I4103, I28K1","수인선, 과천안산선",37.320583,126.806151
751,4814,시우역,I41WS,서해선,일반역,NaN,NaN,37.313767,126.798303


##### 1-b. 결측값 확인

1-b-i. 위경도 결측값 분석

In [301]:
# 역이 노선별로 총 몇 개인지 확인
# 1호선이어도, 구간별로 노선 명칭 다름 (우리가 아는 1호선 = 경원선 + 1호선 + 경인선 + 경부선 + 장항선)
total_station_cnt_by_route = station["노선명"].value_counts().sort_index()
total_station_cnt_by_route

1호선               10
2호선               50
3호선               34
4호선               26
5호선               56
6호선               39
7호선               42
8호선               18
경강선               11
경부선               45
경원선               24
경의중앙선             57
경인선               20
경춘선               24
김포골드라인            10
도시철도 7호선          11
분당선               37
서해선               12
수도권  도시철도 9호선     25
수도권 경량도시철도 신림선    11
수인선               26
신분당선              16
안산과천선             21
에버라인              15
우이신설선             13
의정부               15
인천국제공항선           14
인천지하철 1호선         30
인천지하철 2호선         27
일산선               11
진접선                3
Name: 노선명, dtype: int64

In [302]:
# 역위도, 역경도가 결측값인 역이 노선별로 몇 개인지 확인
station.isnull().sum()

역번호         0
역사명         0
노선번호        0
노선명         0
환승역구분       0
환승노선번호    476
환승노선명     476
역위도       352
역경도       352
dtype: int64

In [303]:
# 역위도, 역경도가 결측값인 역이 노선별로 몇 개인지 확인
null_station_cnt_by_route = station['노선명'][station['역위도'].isnull(
)].value_counts().sort_index()
null_station_cnt_by_route

1호선              10
2호선              50
3호선              34
4호선              26
5호선              56
6호선              39
7호선              42
8호선              18
김포골드라인           10
도시철도 7호선         11
수도권  도시철도 9호선    25
우이신설선            13
의정부              15
진접선               3
Name: 노선명, dtype: int64

1-b-ii. 결측값 채워진 데이터셋 구하기

In [304]:
# 1. 노선별 전체 역 개수가 실제 역 개수와 다름
# 2. 위경도 결측 데이터가 존재하는 노선에 대해 알아본 결과, 서울시 역사마스터 정보와 제대로 합처지지 않은 것으로 보임
# -> 서울시 역사마스터 정보와의 join 필요

# 서울시 역사마스터 데이터 읽어오기
station_seoul = pd.read_csv('train_station_seoul.csv', encoding='euc_kr')

# 기존 데이터셋과 컬럼 이름 맞추기
station_seoul.rename(columns={'역사_ID': '역번호', '호선': '노선명'}, inplace=True)
station.rename(columns={'역위도': '위도', '역경도': '경도'}, inplace=True)

# 서울시 역사마스터 데이터, 위도와 경도가 반대로 들어가 있는 문제 해결
station_seoul.rename(columns={'위도': '경도', '경도': '위도'}, inplace=True)
station_seoul = station_seoul[['역번호', '역사명', '노선명', '위도', '경도']]

# 필요없는 노선 제거
station_seoul = station_seoul[(station_seoul['노선명'] != '장항선')]

station_seoul

,역번호,역사명,노선명,위도,경도
0,9996,미사,5호선,37.560927,127.193877
1,9995,강일,5호선,37.557490,127.175930
2,4929,김포공항,김포골드라인,37.562360,126.801868
3,4928,고촌,김포골드라인,37.601243,126.770345
4,4927,풍무,김포골드라인,37.612488,126.732387
...,...,...,...,...,...
760,154,종로5가,1호선,37.570926,127.001849
761,153,종로3가,1호선,37.570406,126.991847
762,152,종각,1호선,37.570161,126.982923
763,151,시청,1호선,37.565715,126.977088


In [305]:
# 역이 노선별로 몇 개인지 확인
total_station_seoul_cnt_by_route = station_seoul["노선명"].value_counts().sort_index()
total_station_seoul_cnt_by_route

# 확인 결과, 앞서 전체 역정보 데이터셋에 빠진 일부 역이 이 데이터셋에는 포함되어 있음 (역 개수 더 많음)

1호선          10
2호선          50
3호선          34
4호선          26
5호선          58
6호선          39
7호선          53
7호선(인천)       9
8호선          18
9호선          25
9호선(연장)      13
경강선          11
경부선          39
경원선          30
경의중앙선        31
경인선          20
경춘선          19
공항철도1호선      14
과천선           9
김포골드라인       10
분당선          35
서해선          12
수인선          18
신림선          11
신분당선          6
신분당선(연장)      7
신분당선(연장2)     3
안산선          13
에버라인선        15
우이신설선        13
의정부선         15
인천1호선        30
인천2호선        27
일산선          11
중앙선          21
진접선           3
Name: 노선명, dtype: int64

In [306]:
# 역위도, 역경도가 결측값인 역이 노선별로 몇 개인지 확인
station_seoul.isnull().sum()

# 확인 결과, 해당 데이터셋은 위경도 정보가 빠진 것 없이 잘 들어있음 

역번호    0
역사명    0
노선명    0
위도     0
경도     0
dtype: int64

1-b-iii. 노선명 통일

In [307]:
# 둘 중 한 데이터셋에만 있는 노선명 확인
route_name = set(total_station_cnt_by_route.index)
route_name_seoul = set(total_station_seoul_cnt_by_route.index)
sub_route_name = route_name - route_name_seoul
sub_route_name_seoul = route_name_seoul - route_name

print(sub_route_name)
print(sub_route_name_seoul)

{'수도권 경량도시철도 신림선', '인천국제공항선', '인천지하철 2호선', '의정부', '안산과천선', '에버라인', '도시철도 7호선', '수도권  도시철도 9호선', '인천지하철 1호선'}
{'9호선(연장)', '에버라인선', '신분당선(연장2)', '9호선', '의정부선', '인천2호선', '7호선(인천)', '신림선', '중앙선', '공항철도1호선', '안산선', '신분당선(연장)', '과천선', '인천1호선'}


In [308]:
# 노선명 통일
station["노선명"] = station["노선명"].replace({
    "도시철도 7호선": "7호선",
    "수도권  도시철도 9호선": "9호선",
    "수도권 경량도시철도 신림선": "신림선",
    "인천지하철 1호선": "인천1호선",
    "인천지하철 2호선": "인천2호선",
    "인천국제공항선": "공항철도1호선",
    "에버라인": "에버라인선",
    "의정부": "의정부선"
})

# 둘 중 한 데이터셋에만 있는 노선명 재확인
route_name = set(station["노선명"])
sub_route_name = route_name - route_name_seoul
sub_route_name_seoul = route_name_seoul - route_name

print(sub_route_name)
print(sub_route_name_seoul)

# 안산과천선의 경우, 역사명 통일한 이후 안산선 / 과천선으로 나눌 예정

{'안산과천선'}
{'9호선(연장)', '7호선(인천)', '중앙선', '안산선', '신분당선(연장)', '과천선', '신분당선(연장2)'}


1-b-iv. 역사명 통일

In [309]:
# 기존 데이터셋과 역번호 컬럼 데이터 형식 일치시키기
station_seoul = station_seoul.astype({'역번호': 'str'})

# 컬럼 데이터 형식 확인
print(station.dtypes)
print(station_seoul.dtypes)

역번호        object
역사명        object
노선번호       object
노선명        object
환승역구분      object
환승노선번호     object
환승노선명      object
위도        float64
경도        float64
dtype: object
역번호     object
역사명     object
노선명     object
위도     float64
경도     float64
dtype: object


In [310]:
# 둘 중 한 데이터셋에만 있는 역번호, 역사명 확인
station_num = set(station["역번호"])
station_num_seoul = set(station_seoul["역번호"])
sub_station_num = sorted(station_num - station_num_seoul)
sub_station_num_seoul = sorted(station_num_seoul - station_num)

print(sub_station_num)
print(sub_station_num_seoul)

station_name = set(station["역사명"])
station_name_seoul = set(station_seoul["역사명"])
sub_station_name = sorted(station_name - station_name_seoul)
sub_station_name_seoul = sorted(station_name_seoul - station_name)

print(sub_station_name)
print(sub_station_name_seoul)


['0124', '0125', '0126', '0127', '0128', '0129', '0130', '0131', '0132', '0133', '0201', '0202', '0203', '0204', '0205', '0206', '0207', '0208', '0209', '0210', '0211', '0212', '0213', '0214', '0215', '0216', '0217', '0218', '0219', '0220', '0221', '0222', '0223', '0224', '0225', '0226', '0227', '0228', '0229', '0230', '0231', '0232', '0233', '0234', '0235', '0236', '0237', '0238', '0239', '0240', '0241', '0242', '0243', '0319', '0320', '0321', '0322', '0323', '0324', '0325', '0326', '0327', '0328', '0329', '0330', '0331', '0332', '0333', '0334', '0335', '0336', '0337', '0338', '0339', '0340', '0341', '0342', '0343', '0344', '0345', '0346', '0347', '0348', '0349', '0350', '0351', '0352', '0405', '0406', '0408', '0409', '0410', '0411', '0412', '0413', '0414', '0415', '0416', '0417', '0418', '0419', '0420', '0421', '0422', '0423', '0424', '0425', '0426', '0427', '0428', '0429', '0430', '0431', '0432', '0433', '0434', '0510', '0511', '0512', '0513', '0514', '0515', '0516', '0517', '0518',

In [311]:
# 역이름 통일
# 역이름 맨 끝에 등장하는'역'자 제거
station['역사명'] = station['역사명'].str.replace(pat=r'역$', repl=r'', regex=True)
# 서울역은 다시 '역' 추가 처리 
station['역사명'] = station['역사명'].str.replace(pat=r'^서울$', repl=r'서울역', regex=True)

# 결과 다시 확인
station_name = set(station["역사명"])
station_name_seoul = set(station_seoul["역사명"])
sub_station_name = sorted(station_name - station_name_seoul)
sub_station_name_seoul = sorted(station_name_seoul - station_name)

print(sub_station_name)
print(sub_station_name_seoul)

['검단오류', '관악산', '교대(법원·검찰청)', '기흥(백남준아트센터)', '낙성대(강감찬)', '남한산성입구(성남법원·검찰청)', '동대문역사문화공원(DDP)', '동백(용인세브란스)', '동작', '배방', '봉명', '상봉', '서울역(경의선)', '성신여대입구', '수원(분당)', '시민공원', '시우', '시청·용인대', '신창(순천향대)', '신촌(지하)', '쌍용(나사렛대)', '아산', '아시아드경기장', '온수', '온양온천', '왕십리', '용마산(용마폭포공원)', '운동장·송담대(중앙시장)', '운연', '이촌', '전대·에버랜드', '종로3가(탑골공원)', '청량리', '탕정', '하남시청(덕풍∙신장)', '흑석']
['검단오류(검단산업단지)', '관악산(서울대)', '교대(법원.검찰청)', '낙성대', '남한산성입구(성남법원.검찰청)', '동대문역사문화공원', '동백', '둔촌오륜', '봉은사', '삼성중앙', '삼전', '석촌고분', '송파나루', '시민공원(문화창작지대)', '시청.용인대', '아시아드경기장(공촌사거리)', '언주', '용마산', '운동장.송담대', '운연(서창)', '원곡', '이수', '전대.에버랜드', '중앙보훈병원', '하남시청(덕풍-신장)', '한성백제', '흑석(중앙대입구)']


In [312]:
# 특수문자 .으로 통일
station['역사명'] = station['역사명'].str.replace(pat=r'[·∙-]', repl=r'.', regex=True)
station_seoul['역사명'] = station_seoul['역사명'].str.replace(
    pat=r'[·∙-]', repl=r'.', regex=True)

# 수동 맞추기 -> 괄호 생기는 방향으로 (단, 지하/지상 정보, 노선명인 경우 제거)
station["역사명"] = station["역사명"].replace({
    "동작": "동작(현충원)",
    "검단오류": "검단오류(검단산업단지)",
    "관악산": "관악산(서울대)",
    "서울역(경의선)": "서울역",
    "시민공원": "시민공원(문화창작지대)",
    "수원(분당)": "수원",
    "신촌(지하)": "신촌",
    "운연": "운연(서창)",
    "흑석": "흑석(중앙대입구)",
    "아시아드경기장": "아시아드경기장(공촌사거리)"
})

station_seoul["역사명"] = station_seoul["역사명"].replace({
    "낙성대": "낙성대(강감찬)",
    "동대문역사문화공원": "동대문역사문화공원(DDP)",
    "동백": "동백(용인세브란스)",
    "운동장.송담대": "운동장.송담대(중앙시장)",
    "용마산": "용마산(용마폭포공원)"
})

# 결과 다시 확인
station_name = set(station["역사명"])
station_name_seoul = set(station_seoul["역사명"])
sub_station_name = sorted(station_name - station_name_seoul)
sub_station_name_seoul = sorted(station_name_seoul - station_name)

print(sub_station_name)
print(sub_station_name_seoul)

['기흥(백남준아트센터)', '배방', '봉명', '상봉', '성신여대입구', '시우', '신창(순천향대)', '쌍용(나사렛대)', '아산', '온수', '온양온천', '왕십리', '이촌', '종로3가(탑골공원)', '청량리', '탕정']
['둔촌오륜', '봉은사', '삼성중앙', '삼전', '석촌고분', '송파나루', '언주', '원곡', '이수', '중앙보훈병원', '한성백제']


In [313]:
# 안산과천선 -> 안산선 / 과천선
station_name_ansan = set(station_seoul['역사명'][station_seoul['노선명'] == '안산선']) 
station_name_gwacheon = set(
    station_seoul['역사명'][station_seoul['노선명'] == '과천선'])

station.loc[(station['노선명'] == '안산과천선') & (station['역사명'].isin(station_name_ansan)), "노선명"] = "안산선"
station.loc[(station['노선명'] == '안산과천선') & (station['역사명'].isin(station_name_gwacheon)), "노선명"] = "과천선"

# 잘 바뀌었는지 확인
station["노선명"].value_counts().sort_index()

1호선        10
2호선        50
3호선        34
4호선        26
5호선        56
6호선        39
7호선        53
8호선        18
9호선        25
경강선        11
경부선        45
경원선        24
경의중앙선      57
경인선        20
경춘선        24
공항철도1호선    14
과천선         8
김포골드라인     10
분당선        37
서해선        12
수인선        26
신림선        11
신분당선       16
안산선        13
에버라인선      15
우이신설선      13
의정부선       15
인천1호선      30
인천2호선      27
일산선        11
진접선         3
Name: 노선명, dtype: int64

##### 1-c. 두 데이터셋 하나로 결합

1-c-i. 위경도값 하나로 합치기

In [314]:
# 일단 하나로 합쳐보기
station_result = pd.merge(station, station_seoul, on=['역사명', '노선명'], how='outer')
station_result

,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,역번호_y,위도_y,경도_y
0,3110,계양,S2801,인천1호선,환승역,S2801+I28A1,공항철도,37.571539,126.736319,3110,37.571449,126.735780
1,3111,귤현,S2801,인천1호선,일반역,-,-,37.566362,126.742498,3111,37.566379,126.742654
2,3112,박촌,S2801,인천1호선,일반역,-,-,37.553525,126.744946,3112,37.553703,126.745077
3,3113,임학,S2801,인천1호선,일반역,-,-,37.545058,126.738642,3113,37.545059,126.738665
4,3114,계산,S2801,인천1호선,일반역,-,-,37.543243,126.728436,3114,37.543238,126.728128
...,...,...,...,...,...,...,...,...,...,...,...,...
822,NaN,옥수,NaN,경원선,NaN,NaN,NaN,NaN,NaN,1011,37.540446,127.018672
823,NaN,한남,NaN,경원선,NaN,NaN,NaN,NaN,NaN,1010,37.529430,127.009169
824,NaN,서빙고,NaN,경원선,NaN,NaN,NaN,NaN,NaN,1009,37.519594,126.988537
825,NaN,이촌(국립중앙박물관),NaN,경원선,NaN,NaN,NaN,NaN,NaN,1008,37.522427,126.973406


In [315]:
# x, y에 대해, 둘 중 한 곳에 없는 위경도값이 각각 몇 개인지 count
lat_in_only_x = station_result[station_result["위도_y"].isnull(
) & station_result["위도_x"].notnull()]
lng_in_only_x = station_result[station_result["경도_y"].isnull(
) & station_result["경도_x"].notnull()]

lat_in_only_x_cnt = len(lat_in_only_x)
lng_in_only_x_cnt = len(lng_in_only_x)

print("x에만 있는 위경도값 개수 :", lat_in_only_x_cnt, lng_in_only_x_cnt)

lat_in_only_y = station_result[station_result["위도_x"].isnull(
) & station_result["위도_y"].notnull()]
lng_in_only_y = station_result[station_result["경도_x"].isnull(
) & station_result["경도_y"].notnull()]

lat_in_only_y_cnt = len(lat_in_only_y)
lng_in_only_y_cnt = len(lng_in_only_y)

print("y에만 있는 위경도값 개수 :", lat_in_only_y_cnt, lng_in_only_y_cnt)

# 한쪽 위경도가 NaN이면, 반대쪽 위경도 넣어주기
station_result["위도_x"] = station_result["위도_x"].fillna(station_result["위도_y"])
station_result["위도_y"] = station_result["위도_y"].fillna(station_result["위도_x"])
station_result["경도_x"] = station_result["경도_x"].fillna(station_result["경도_y"])
station_result["경도_y"] = station_result["경도_y"].fillna(station_result["경도_x"])

# 위경도 데이터의 오차 계산
station_result["위도차"] = (station_result["위도_x"] - station_result["위도_y"]).abs()
station_result["경도차"] = (station_result["경도_x"] - station_result["경도_y"]).abs()

x에만 있는 위경도값 개수 : 66 66
y에만 있는 위경도값 개수 : 419 419


In [316]:
# 위도 오차값 확인
lat_diff_mean = station_result["위도차"].mean()
print(lat_diff_mean)

# 평균보다 오차 큰 행 확인
bigger_lat_diff = station_result[station_result["위도차"] >= lat_diff_mean]
bigger_lat_diff

0.6563686820821601


,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,역번호_y,위도_y,경도_y,위도차,경도차
113,D007,강남,I11D1,신분당선,도시철도 환승역,I11D1,수도권 광역철도 신분당선,127.027879,37.497385,4307,37.496837,127.028104,89.531042,89.530719
114,D008,양재(서초구청),I11D1,신분당선,도시철도 환승역,I11D1,수도권 광역철도 신분당선,127.034099,37.484454,4308,37.483809,127.034653,89.550290,89.550199
115,D009,양재시민의숲(매헌),I11D1,신분당선,도시철도 일반역,NaN,NaN,127.038420,37.470964,4309,37.470023,127.038420,89.568397,89.567456
116,D010,청계산입구,I11D1,신분당선,도시철도 일반역,NaN,NaN,127.054340,37.448715,4310,37.447211,127.055664,89.607129,89.606949
117,D011,판교,I11D1,신분당선,도시철도 일반역,I11D1,수도권 광역철도 신분당선,127.111211,37.395715,4311,37.394761,127.112217,89.716450,89.716502
118,D012,정자,I11D1,신분당선,도시철도 환승역,I11D1,수도권 광역철도 신분당선,127.108386,37.368210,4312,37.367098,127.108403,89.741288,89.740193
710,1886,송도,I28K1,수인선,일반역,NaN,NaN,34.429705,126.654401,1886,37.428514,126.657772,2.998809,0.003371


In [317]:
# 경도 오차값 확인
lng_diff_mean = station_result["경도차"].mean()
print(lng_diff_mean)

# 평균보다 오차 큰 행 확인
bigger_lng_diff = station_result[station_result["경도차"] >= lng_diff_mean]
bigger_lng_diff

0.656387570673301


,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,역번호_y,위도_y,경도_y,위도차,경도차
54,3225,남동구청,S2802,인천2호선,일반역,-,-,37.448179,123.736581,3225,37.448161,126.736939,0.000018,3.000358
113,D007,강남,I11D1,신분당선,도시철도 환승역,I11D1,수도권 광역철도 신분당선,127.027879,37.497385,4307,37.496837,127.028104,89.531042,89.530719
114,D008,양재(서초구청),I11D1,신분당선,도시철도 환승역,I11D1,수도권 광역철도 신분당선,127.034099,37.484454,4308,37.483809,127.034653,89.550290,89.550199
115,D009,양재시민의숲(매헌),I11D1,신분당선,도시철도 일반역,NaN,NaN,127.038420,37.470964,4309,37.470023,127.038420,89.568397,89.567456
116,D010,청계산입구,I11D1,신분당선,도시철도 일반역,NaN,NaN,127.054340,37.448715,4310,37.447211,127.055664,89.607129,89.606949
117,D011,판교,I11D1,신분당선,도시철도 일반역,I11D1,수도권 광역철도 신분당선,127.111211,37.395715,4311,37.394761,127.112217,89.716450,89.716502
118,D012,정자,I11D1,신분당선,도시철도 환승역,I11D1,수도권 광역철도 신분당선,127.108386,37.368210,4312,37.367098,127.108403,89.741288,89.740193


In [318]:
# 113 ~ 118, 위도_x와 경도_x가 뒤바뀌어 들어감
# 송도, (위도_x, 경도_x) 전남 해남에 위치 
# 정자, (위도_x, 경도_x) 황해에 위치
# -> 위경도 데이터의 경우, x를 y로 덮어쓰면 될 것 같음
# 이를 증명하기 위해, 위의 데이터들의 위도차/경도차가 해결된 뒤, 
# 위경도차의 max값을 각각 확인하여, 거리상의 오차가 어느 정도일지 확인해볼 것

station_result.loc[[113, 114, 115, 116, 117, 118, 710, 54], ['위도차', '경도차']] = 0.0

In [319]:
# 위도 오차값 평균 재확인
lat_diff_mean = station_result["위도차"].mean()
print(lat_diff_mean)

# 평균보다 오차 큰 행 확인
bigger_lat_diff = station_result[station_result["위도차"] >= lat_diff_mean]
bigger_lat_diff.sort_values('위도차').tail()

0.00016307176662614506


,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,역번호_y,위도_y,경도_y,위도차,경도차
740,1952,화정,I4106,일산선,일반역,NaN,NaN,37.637837,126.832503,1952,37.634592,126.832650,0.003245,0.000147
758,4815,원시,I41WS,서해선,일반역,NaN,NaN,37.305947,126.788297,4815,37.302371,126.786691,0.003576,0.001606
44,3215,인천가좌,S2802,인천2호선,일반역,-,-,37.493762,126.675465,3215,37.489700,126.675208,0.004062,0.000257
83,A01,서울역,I28A1,공항철도1호선,환승역,S1101+S1104+I4108,서울 도시철도 1호선+서울 도시철도 4호선+경의선,37.549118,126.970464,4201,37.553247,126.969769,0.004129,0.000695
563,1268,화전,I4108,경의중앙선,일반역,NaN,NaN,37.637837,126.832503,1268,37.602888,126.868387,0.034949,0.035884


In [320]:
# 경도 오차값 평균 재확인
lng_diff_mean = station_result["경도차"].mean()
print(lng_diff_mean)

# 평균보다 오차 큰 행 확인
bigger_lng_diff = station_result[station_result["경도차"] >= lng_diff_mean]
bigger_lng_diff.sort_values('경도차').tail()

0.00017913939902904088


,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,역번호_y,위도_y,경도_y,위도차,경도차
36,3207,검암,S2802,인천2호선,환승역,S2802+I28A1,공항철도,37.568611,126.671221,3207,37.568660,126.675687,0.000049,0.004466
468,1502,이매,I41K5,경강선,환승역,I28K1,분당선,37.394736,127.123048,1502,37.394655,127.127819,0.000081,0.004771
692,1876,야목,I28K1,수인선,일반역,NaN,NaN,37.261011,126.884290,1876,37.264179,126.879483,0.003168,0.004807
96,A11,인천공항2터미널,I28A1,공항철도1호선,일반역,NaN,NaN,37.458269,126.427536,4215,37.460699,126.441442,0.002430,0.013906
563,1268,화전,I4108,경의중앙선,일반역,NaN,NaN,37.637837,126.832503,1268,37.602888,126.868387,0.034949,0.035884


In [321]:
# 화전, (위도_x, 경도_x) 덕양구청에 위치, 도보 1시간 30분의 오차
# 그 다음으로 오차가 큰 인천공항2터미널과 서울역의 경우 역이 워낙 커서 생기는 일, 문제 없음
# 즉, x와 y 두 곳에 있는 값들에 대해서는 이상값 검토가 끝남 
# 둘 다에 있는 값의 경우 그냥 y쪽 값으로 덮어쓰기
# 둘 중 한 곳에만 있는 위경도값들은 위에서 따로 저장해 둠, 
# 이후 지도를 보며 재검토할 예정  

# y에 없는 값들 x에서 가져와 채우기
station_result['위도_y'] = station_result['위도_y'].fillna(station_result['위도_x'])
station_result['경도_y'] = station_result['경도_y'].fillna(station_result['경도_x'])

# 개수 재확인
null_lat_in_y = station_result[station_result["위도_y"].isnull()]
null_lng_in_y = station_result[station_result["경도_y"].isnull()]
null_lat_in_y_cnt = len(null_lat_in_y)
null_lng_in_y_cnt = len(null_lng_in_y)

print("y에 있는 위경도 결측값 개수 :", null_lat_in_y_cnt, null_lng_in_y_cnt)

null_lat_in_y 

y에 있는 위경도 결측값 개수 : 3 3


,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,위도_x,경도_x,역번호_y,위도_y,경도_y,위도차,경도차
107,S120,성신여대입구,L11UI,우이신설선,도시철도 환승역,I1104,수도권 광역철도 4호선,NaN,NaN,NaN,NaN,NaN,NaN,NaN
294,0534,종로3가(탑골공원),S1105,5호선,도시철도 환승역,I1103+I1101,수도권 광역철도 3호선+수도권 광역철도 1호선,NaN,NaN,NaN,NaN,NaN,NaN,NaN
394,0736,총신대입구(이수),S1107,7호선,도시철도 환승역,I1104,수도권 광역철도 4호선,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [322]:
# 성신여대입구, 종로3가, 총신대입구에 대한 위경도 정보 y쪽에 수동으로 넣어주기 

station_result.loc[107, "위도_y"] = 37.592359
station_result.loc[107, "경도_y"] = 127.016551 

station_result.loc[294, "위도_y"] = 37.572571
station_result.loc[294, "경도_y"] = 126.990293

station_result.loc[394, "위도_y"] = 37.485263
station_result.loc[394, "경도_y"] = 126.981609

# 개수 재확인
null_lat_in_y = station_result[station_result["위도_y"].isnull()]
null_lng_in_y = station_result[station_result["경도_y"].isnull()]
null_lat_in_y_cnt = len(null_lat_in_y)
null_lng_in_y_cnt = len(null_lng_in_y)

print("y에 있는 위경도 결측값 개수 :", null_lat_in_y_cnt, null_lng_in_y_cnt)

y에 있는 위경도 결측값 개수 : 0 0


In [323]:
# 위도_x, 경도_x 삭제 및 위도_y와 경도_y의 이름 변경
station_result.drop(columns=["위도_x", "경도_x", "위도차", "경도차"], inplace=True)
station_result.rename(columns={'위도_y': '위도', '경도_y': '경도'}, inplace=True)

station_result

,역번호_x,역사명,노선번호,노선명,환승역구분,환승노선번호,환승노선명,역번호_y,위도,경도
0,3110,계양,S2801,인천1호선,환승역,S2801+I28A1,공항철도,3110,37.571449,126.735780
1,3111,귤현,S2801,인천1호선,일반역,-,-,3111,37.566379,126.742654
2,3112,박촌,S2801,인천1호선,일반역,-,-,3112,37.553703,126.745077
3,3113,임학,S2801,인천1호선,일반역,-,-,3113,37.545059,126.738665
4,3114,계산,S2801,인천1호선,일반역,-,-,3114,37.543238,126.728128
...,...,...,...,...,...,...,...,...,...,...
822,NaN,옥수,NaN,경원선,NaN,NaN,NaN,1011,37.540446,127.018672
823,NaN,한남,NaN,경원선,NaN,NaN,NaN,1010,37.529430,127.009169
824,NaN,서빙고,NaN,경원선,NaN,NaN,NaN,1009,37.519594,126.988537
825,NaN,이촌(국립중앙박물관),NaN,경원선,NaN,NaN,NaN,1008,37.522427,126.973406
